In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print('GPU acceleration is available and will be used :-)')
else:
    DEVICE = torch.device("cpu")
    print('GPU is not available. If you are using Google Colab, change the runtime to GPU, otherwise training will '
          'take too long.')

GPU is not available. If you are using Google Colab, change the runtime to GPU, otherwise training will take too long.


In [75]:
!sh download_data.sh

--2021-01-27 21:16:43--  https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.251.30
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.251.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209850483 (200M) [application/json]
Saving to: ‘./data/personachat_self_original.json.1’

personachat_self_or 100%[===================>] 200.13M  5.70MB/s    in 36s     

2021-01-27 21:17:19 (5.60 MB/s) - ‘./data/personachat_self_original.json.1’ saved [209850483/209850483]



In [65]:
from data_util import generate_train_validation_test_files

generate_train_validation_test_files()

0 lines removed
Train set 131,438
0 lines removed
Test set 7,801


In [4]:
from data_util import DataWrapper

# Dataset objects
dw = DataWrapper()
train_ds, val_ds, test_ds = dw.get_datasets(
    train_size=99,
    val_size=99,
    use_glove=True
)
print(f'Train set size: {len(train_ds):,}')
print(f'Validation set size: {len(val_ds):,}')
print('Vocab size: ', dw.vocab_size)

Train set size: 99
Validation set size: 99
Vocab size:  500


In [5]:
# DataLoader objects
train_iter, val_iter, test_iter = dw.get_dataloaders(
    train_ds, val_ds, test_ds,
    batch_size=2400,
    device=DEVICE
)

x = next(iter(train_iter))
print('Example \n-------')
print(x.src[0])
print(x.src[1])

Example 
-------
tensor([[  2,  34,  10,  21,  14,   6,  48,   8,   4,  27, 116, 119,  11,  12,
          43, 189, 188,  11, 229,  24, 224,   9,   6, 215,  80,  67, 197,   5,
         101,  16, 118,  26,  13,  40,  78,   9,   4,  15,  20,  25,  13, 146,
           4,  22,  11,  12, 142,  53,  43, 293,   9,   4,  51, 278, 208,  70,
           4,  15,  31, 159, 249, 101,   9,  23,  54, 345,   5,  70,   4,  57,
          24, 323,  73,   4, 351, 302,  24, 301, 341, 311,  20, 374,  54,  98,
           5,  12,   6,  18,   7,  40, 223,  53,  45,  26, 163,   8,  28,  12,
          31,   5,  32,   4,  12,  18,   7,  40, 147, 161,  23,  16,  86,   4,
         106, 389,   9,  30,  16,  39,  40, 147,  11, 106,   8,  28, 112,  18,
          11, 484, 265, 478, 481,   5,  12,   6,  18,  79,  40, 459,   8,  28,
          22, 445,  53, 468,  19, 444,   9,  12,   6,  18, 105,   0,  25,  41,
           8,   4, 420,   4,  15,  82,  11,  12,  43, 142,   9,   4,  15,  82,
          11,  68,  62,   5,  30,  

/Users/paulabartabajo/miniconda3/envs/chatbot_py3.6/lib/python3.6/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/paulabartabajo/miniconda3/envs/chatbot_py3.6/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [7]:
%autoreload 2
from model import Seq2seqRNN, count_parameters

vocab_size = dw.vocab_size
embedding_dim = dw.embedding_dim
hidden_dim = 256
n_layers = 3
n_directions_encoder = 2
model = Seq2seqRNN(vocab_size,
                   embedding_dim,
                   hidden_dim,
                   n_layers,
                   n_directions_encoder,
                   dropout=0.2,
                   pretrained_embeddings=dw.embeddings,
                   freeze_embeddings=False)

print(f'The model has {count_parameters(model):,} parameters')

The model has 5,484,868 parameters


In [15]:
from train import Seq2seqRNNTrainer

trainer = Seq2seqRNNTrainer(model,
                            train_iter,
                            val_iter,
                            learning_rate=3e-4,
                            pad_token_id=dw.pad_token_id,
                            gradient_clip=99999,
                            teacher_forcing=0.5)

In [ ]:
n_epochs = 10
trainer.train(n_epochs)

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

Epoch: 000, Train loss: 4.8019, Val loss: 4.2843, Train ppl: 121.7, Val ppl: 72.6


  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

Epoch: 001, Train loss: 4.7110, Val loss: 4.2226, Train ppl: 111.2, Val ppl: 68.2


  0%|          | 0/99 [00:00<?, ?it/s]